In [ ]:
import logging
from abc import abstractmethod
from collections import defaultdict, namedtuple
from enum import Enum
from pprint import pprint

import numpy as np

In [ ]:
# test_input = """
# broadcaster -> a, b, c
# %a -> b
# %b -> c
# %c -> inv
# &inv -> a
# """

test_input = """
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
"""

In [ ]:
logger = logging.getLogger(__name__)
Signal = namedtuple("Signal", ["source", "destination", "value"])
Module = namedtuple("Module", ["name", "type", "connections"])


def make_module(name: str, connections: list[str]) -> Module:
    if name.startswith("%"):
        return Module(name[1:], "not", connections)
    elif name.startswith("&"):
        return Module(name[1:], "nand", connections)
    else:
        return Module(name, "signal", connections)

In [ ]:
def part1(text_input: str) -> int | str:
    modules: dict[str, Module] = {}
    # Memory for each nand module
    nand_memory: dict[str, dict[str, int]] = {}
    # On / Off for each not module
    not_memory: dict[str, int] = {}

    modules["button"] = Module("button", "signal", ["broadcaster"])
    modules["output"] = Module("button", "signal", [])
    modules["rx"] = Module("button", "signal", [])
    for line in text_input.strip().split("\n"):
        module, connections = map(str.strip, line.split("->"))
        connections = list(map(str.strip, connections.split(",")))
        module = make_module(module, connections)
        modules[module.name] = module
        if module.type == "nand":
            nand_memory[module.name] = {}
        if module.type == "not":
            not_memory[module.name] = 0
    # Make memory
    for name, module in modules.items():
        if module.type == "nand":
            for name_, module_ in modules.items():
                if name in module_.connections:
                    nand_memory[name][name_] = 0

    counts = [0, 0]
    for i in range(1000):
        logger.info(f"--- Step {i+1} ---")
        signals = [Signal("button", "broadcaster", 0)]
        while signals:
            signal = signals.pop(0)
            counts[signal.value] += 1
            logger.info(
                f"{signal.source} {['-low', '-high'][signal.value]}-> {signal.destination}"
            )
            source, destination = modules[signal.source], modules[signal.destination]
            if destination.type == "signal":
                # Forward signal to destinations
                for connection in destination.connections:
                    signals.append(Signal(destination.name, connection, signal.value))
            elif destination.type == "not":
                if signal.value == 0:
                    emit = not not_memory[destination.name]
                    not_memory[destination.name] = emit
                    for connection in destination.connections:
                        signals.append(Signal(destination.name, connection, emit))
            elif destination.type == "nand":
                # Update memory
                nand_memory[destination.name][source.name] = signal.value
                # logger.info(f"{destination.name} : {nand_memory[destination.name]}")
                emit = not all(nand_memory[destination.name].values())
                for connection in destination.connections:
                    signals.append(Signal(destination.name, connection, emit))

    return counts[0] * counts[1]

In [ ]:
part1(test_input)

In [ ]:
import graphviz
def draw_graph(text_input: str):
    dot = graphviz.Digraph()
    nodes = set()
    link = {}
    for line in text_input.strip().split("\n"):
        module, connections = map(str.strip, line.split("->"))
        connections = list(map(str.strip, connections.split(",")))
        module = make_module(module, connections)
        dot.node(module.name)
        for b in connections:
            dot.edge(module.name, b)
    return dot

In [ ]:
import requests
year, day = 2023, 20
url = f"https://adventofcode.com/{year}/day/{day}/input"
with open("../../session.txt") as f:
    session = f.read().strip()
cookies = {"session": session}
response = requests.get(url, cookies=cookies)
day_input = response.text.strip()

In [ ]:
#draw_graph(day_input)

In [ ]:
def part2(text_input: str) -> int | str:
    modules: dict[str, Module] = {}
    # Memory for each nand module
    nand_memory: dict[str, dict[str, int]] = {}
    # On / Off for each not module
    not_memory: dict[str, int] = {}

    modules["button"] = Module("button", "signal", ["broadcaster"])
    modules["output"] = Module("button", "signal", [])
    modules["rx"] = Module("button", "signal", [])
    for line in text_input.strip().split("\n"):
        module, connections = map(str.strip, line.split("->"))
        connections = list(map(str.strip, connections.split(",")))
        module = make_module(module, connections)
        modules[module.name] = module
        if module.type == "nand":
            nand_memory[module.name] = {}
        if module.type == "not":
            not_memory[module.name] = 0
    # Make memory
    for name, module in modules.items():
        if module.type == "nand":
            for name_, module_ in modules.items():
                if name in module_.connections:
                    nand_memory[name][name_] = 0

    counts = [0, 0]
    antepenultimates = []
    i = 0
    while len(antepenultimates) < 4:
        i += 1
        logger.info(f"--- Step {i+1} ---")
        signals = [Signal("button", "broadcaster", 0)]
        while signals:
            signal = signals.pop(0)
            if signal.source in ["sz", "gc", "cm", "xf"] and signal.value == 1:
                antepenultimates.append(i)
            counts[signal.value] += 1
            logger.info(
                f"{signal.source} {['-low', '-high'][signal.value]}-> {signal.destination}"
            )
            source, destination = modules[signal.source], modules[signal.destination]
            if destination.type == "signal":
                # Forward signal to destinations
                for connection in destination.connections:
                    signals.append(Signal(destination.name, connection, signal.value))
            elif destination.type == "not":
                if signal.value == 0:
                    emit = not not_memory[destination.name]
                    not_memory[destination.name] = emit
                    for connection in destination.connections:
                        signals.append(Signal(destination.name, connection, emit))
            elif destination.type == "nand":
                # Update memory
                nand_memory[destination.name][source.name] = signal.value
                # logger.info(f"{destination.name} : {nand_memory[destination.name]}")
                emit = not all(nand_memory[destination.name].values())
                for connection in destination.connections:
                    signals.append(Signal(destination.name, connection, emit))

    return math.lcm(*antepenultimates)

In [ ]:
part2(day_input)